In [9]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

In [11]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [12]:
#in real world, this would be an API call to a stock price service
@tool
def get_stock_price(symbol: str) -> float:
    '''Return the current price of a stock given the symbol
    :param symbol: stock symbol
    :return: current price of the stock
    '''
    return {
        "MSFT": 300.0,
        "AAPL": 150.0,
        "AMZN": 3500.0,
        "RIL": 2000.0,
    }.get(symbol, 0.0)

tools = [get_stock_price]

llm = init_chat_model("llama-3.3-70b-versatile",model_provider="groq")
llm_with_tools = llm.bind_tools(tools)

In [17]:
def chatbot(state: State) -> State:
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot_node")
builder.add_conditional_edges("chatbot_node", tools_condition)
# builder.add_edge(tools_condition, "tools")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [18]:
from IPython.display import Markdown
Markdown(f"```mermaid\n{graph.get_graph().draw_mermaid()}\n```")

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot_node(chatbot_node)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot_node;
	chatbot_node -.-> __end__;
	chatbot_node -.-> tools;
	tools --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [25]:
state = graph.invoke({"messages": [{"role": "user", 
    "content": 
    "I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be the total cost?"}]})

print(state["messages"][-1].content)

300.0


In [22]:
state

{'messages': [HumanMessage(content='Who invented the telephone?', additional_kwargs={}, response_metadata={}, id='949eb403-79f2-42c3-9730-9a4799f76e4f'),
  AIMessage(content='The invention of the telephone is often credited to Alexander Graham Bell, who patented the first practical telephone in 1876. However, there is some controversy over whether Bell was the sole inventor, as other individuals, such as Elisha Gray and Antonio Meucci, were also working on similar devices around the same time. Nonetheless, Bell is widely recognized as the inventor of the first practical telephone.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 263, 'total_tokens': 343, 'completion_time': 0.195230371, 'completion_tokens_details': None, 'prompt_time': 0.059818952, 'prompt_tokens_details': None, 'queue_time': 0.070106945, 'total_time': 0.255049323}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'service_tier': 'on_dema